In [1]:
library(MatchIt)
library(fiftystater)
library(maptools)
library(rgeos)
library(rgdal)
library(dplyr)
library(knitr)
library(pacman)
library(optmatch)
library(data.table)
library(sjstats)
library(sjmisc)
library(survey)

Loading required package: sp
Checking rgeos availability: TRUE
rgeos version: 0.3-28, (SVN revision 572)
 GEOS runtime version: 3.5.0-CAPI-1.9.0 r4084 
 Linking to sp version: 1.3-1 
 Polygon checking: TRUE 

rgdal: version: 1.3-4, (SVN revision 766)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.11.4, released 2016/01/25
 Path to GDAL shared files: /usr/share/gdal
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.8.0, 6 March 2012, [PJ_VERSION: 480]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.3-1 

Attaching package: ‘dplyr’

The following objects are masked from ‘package:rgeos’:

    intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: survival
The optmatch package has an academic license. Enter relaxinfo() for more info

In [2]:
# Race indicates whether match is for specific sub-population, or all zip codes
race <- "all"

# Variable to intervene on
treatmentVar <- "MedianFamilyIncome"

#Match on all variables except for treatment var
controls <- list('MedianFamilyIncome','no_college','lapophalfshare','category_hotdogs_frac')
controls<- controls[controls!=treatmentVar]


df <- read.csv(paste('../precomputed_matches/', treatmentVar , '.csv',sep = ""))
df$Treatment <- df$Treatment == "False"
df$LogMedianFamilyIncome <- log(df$MedianFamilyIncome)


## Run genetic matching algorithm

In [3]:
set.seed(42)
# Set caliper=XX if better matches are needed
formula <- paste("Treatment ~ " , paste(controls, collapse = ' + '))
match.it <- matchit(as.formula(formula),data = df, method="genetic",pop.size=1000,cluster = c("localhost", "localhost", "localhost","localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost","localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost", "localhost" , "localhost"),fit.func='qqmean.mean')



Loading required namespace: rgenoud
Loading required namespace: parallel


Initializing Cluster


Sun Jul 26 14:25:23 2020
Domains:
 0.000000e+00   <=  X1   <=    1.000000e+03 
 0.000000e+00   <=  X2   <=    1.000000e+03 
 0.000000e+00   <=  X3   <=    1.000000e+03 
 0.000000e+00   <=  X4   <=    1.000000e+03 

Data Type: Floating Point
Operators (code number, name, population) 
	(1) Cloning........................... 	122
	(2) Uniform Mutation.................. 	125
	(3) Boundary Mutation................. 	125
	(4) Non-Uniform Mutation.............. 	125
	(5) Polytope Crossover................ 	125
	(6) Simple Crossover.................. 	126
	(7) Whole Non-Uniform Mutation........ 	125
	(8) Heuristic Crossover............... 	126
	(9) Local-Minimum Crossover........... 	0

SOFT Maximum Number of Generations: 100
Maximum Nonchanging Generations: 4
Population size       : 1000
Convergence Tolerance: 1.000000e-03

Not Using the BFGS Derivative Based Optimizer on the Best Individual Each Generation.
Not Checking Gradients before Stopping.
Using Out of Bounds In

In [4]:
logResults <- function(mit,saveName){
    a <- summary(mit,addlvariables=df[treatmentVar])
    mdata <- match.data(mit)
    summaryCounts <- a$nn
    summaryBalance <- a$sum.matched
    summaryBalance["STD Mean Difference"] <- summaryBalance[4]/summaryBalance[3]
    
    write.csv(mdata,paste(list('log/',saveName,race,"_",treatmentVar,'_data.csv'),collapse=''))
    write.csv(summaryBalance,paste(list('log/',saveName,race,"_",treatmentVar,'_summaryCount.csv'),collapse=''))
    write.csv(summaryCounts,paste(list('log/',saveName,race,"_",treatmentVar,'_summaryBalance.csv'),collapse=''))
}



In [5]:
logResults(match.it,'')

In [6]:
a <- summary(match.it,addlvariables=df[treatmentVar])

In [7]:
mdata <- match.data(match.it)

In [8]:
x = kable(a$nn, digits = 2, align = 'c', caption = 'Sample sizes')
x
write(x,file=paste(list('table_',race,"_",treatmentVar,'.csv'),collapse=''))
x = kable(a$nn, digits = 2, align = 'c',format='latex', caption = 'Sample sizes')
write(x,file=paste(list('table_',race,"_",treatmentVar,'.csv'),collapse=''),append=TRUE)



|          | Control | Treated |
|:---------|:-------:|:-------:|
|All       |  2319   |  2321   |
|Matched   |   215   |  2321   |
|Unmatched |  2104   |    0    |
|Discarded |    0    |    0    |

In [9]:
kab <- a$sum.matched[c(1,2,3,4)]
kab["STD Mean Difference"] <- kab[4]/kab[3]
x = kable(kab, digits = 2, align = 'c', 
      caption = 'Summary of balance for matched data')
x
write(x,file=paste(list('table_',race,"_",treatmentVar,'.csv'),collapse=''),append=TRUE)
x = kable(kab, digits = 2, align = 'c', 
      caption = 'Summary of balance for matched data',format='latex')
write(x,file=paste(list('table_',race,"_",treatmentVar,'.csv'),collapse=''),append=TRUE)





|                      | Means Treated | Means Control | SD Control | Mean Diff | STD Mean Difference |
|:---------------------|:-------------:|:-------------:|:----------:|:---------:|:-------------------:|
|distance              |     0.91      |     0.91      |    0.18    |   0.00    |        0.01         |
|no_college            |     0.47      |     0.48      |    0.13    |   -0.02   |        -0.11        |
|lapophalfshare        |     0.71      |     0.70      |    0.22    |   0.01    |        0.04         |
|category_hotdogs_frac |     0.04      |     0.04      |    0.04    |   0.00    |        -0.06        |
|MedianFamilyIncome    |   115443.80   |   48357.34    |  6021.64   | 67086.46  |        11.14        |

In [10]:
df.match <- match.data(match.it)[1:ncol(df)]
mdata <- match.data(match.it)


In [11]:
count(df.match)

n
2536


## Calculate mean effect sizes for all outcomes

In [12]:
a<-weighted.mean(mdata$Fresh_FV[mdata$Treatment == 1], mdata$weights[mdata$Treatment==1]) 
b<-weighted.mean(mdata$Fresh_FV[mdata$Treatment==0], mdata$weights[mdata$Treatment==0])


c(((a-b)/b)*100.0,a,b)
change1 = ((a-b)/b)*100.0
a1 = a
b1 = b

[1] 7.3830321 0.6828644 0.6359146

In [13]:
a<-weighted.mean(mdata$fast_food[mdata$Treatment == 1], mdata$weights[mdata$Treatment==1]) 
b<-weighted.mean(mdata$fast_food[mdata$Treatment==0], mdata$weights[mdata$Treatment==0])

c(((a-b)/b)*100.0,a,b)
change2 = ((a-b)/b)*100.0
a2 = a
b2 = b

[1] -9.3661409  0.3124393  0.3447269

In [14]:
a<-weighted.mean(mdata$soda[mdata$Treatment == 1], mdata$weights[mdata$Treatment==1]) 
b<-weighted.mean(mdata$soda[mdata$Treatment==0], mdata$weights[mdata$Treatment==0])

c(((a-b)/b)*100.0,a,b)
change3 = ((a-b)/b)*100.0
a3 = a
b3 = b

[1] -7.06383463  0.02937768  0.03161061

In [15]:
a<-weighted.mean(mdata$bmi[mdata$Treatment == 1], mdata$weights[mdata$Treatment==1]) 
b<-weighted.mean(mdata$bmi[mdata$Treatment==0], mdata$weights[mdata$Treatment==0])

c(((a-b)/b)*100.0,a,b)
change4 = ((a-b)/b)*100.0
change5= a-b
a4 = a
b4 = b

[1]  0.6890831 27.3622946 27.1750360

In [16]:
a<-weighted.mean(mdata$ow[mdata$Treatment == 1], mdata$weights[mdata$Treatment==1]) 
b<-weighted.mean(mdata$ow[mdata$Treatment==0], mdata$weights[mdata$Treatment==0])

c(((a-b)/b)*100.0,a,b)
change6 = ((a-b)/b)*100.0
a6 = a
b6 = b

[1] 2.9035032 0.6133319 0.5960262

## Run N=1000 bootstrapped CI

In [17]:
mat <- setDT(as.data.frame(match.it$match.matrix), keep.rownames = TRUE)[]
exp <- data.frame(matrix(ncol = 6, nrow = 0))
x <- c("Fresh_FV", "fast_food", "soda","bmi","bmi_abs","ow")
colnames(exp) <- x

for (i in 1:1000){
    index <- sample(1:nrow(mat), nrow(mat), replace=TRUE)
    matSample <- mat[index,]
    df_control <- df[as.numeric(as.character(unlist(matSample[,2], use.names=FALSE))),]
    df_treatment <- df[as.numeric(as.character(unlist(matSample[,1], use.names=FALSE))),]

    a<-mean(df_treatment$Fresh_FV)
    b<-mean(df_control$Fresh_FV)
    changeFV = ((a-b)/b)*100.0
    
    a<-mean(df_treatment$fast_food)
    b<-mean(df_control$fast_food)
    changeFF = ((a-b)/b)*100.0
    
    a<-mean(df_treatment$soda)
    b<-mean(df_control$soda)
    changeSoda = ((a-b)/b)*100.0 
    
    a_<-mean(df_treatment$bmi)
    b_<-mean(df_control$bmi)
    changeBMI = ((a_-b_)/b_)*100.0 
        
    a<-mean(df_treatment$ow)
    b<-mean(df_control$ow)
    changeOW = ((a-b)/b)*100.0 
    
    exp[i, ] <- list(changeFV, changeFF, changeSoda,changeBMI,a_-b_,changeOW)
    message(i)
}

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Calculate bootstrapped p-values  

In [ ]:
upper = quantile(exp$Fresh_FV, c(0.025,0.975))[[2]] - change1
lower = change1 - quantile(exp$Fresh_FV, c(0.025,0.975))[[1]]
p_value = sum(exp$Fresh_FV <= 0)/1000.0
if (change1 < 0){
    p_value = sum(exp$Fresh_FV >= 0)/1000.0
}

write(paste(list(race,treatmentVar,'Fresh_FV',change1,upper,lower,a1,b1,p_value),collapse=', '),file="main_outcomes.csv",append=TRUE)




In [ ]:
upper = quantile(exp$fast_food, c(0.025,0.975))[[2]] - change2
lower = change2 - quantile(exp$fast_food, c(0.025,0.975))[[1]]
p_value = sum(exp$fast_food >= 0)/1000.0
if (change2 > 0){
    p_value = sum(exp$fast_food <= 0)/1000.0
}

write(paste(list(race,treatmentVar,'fast_food',change2,upper,lower,a2,b2,p_value),collapse=', '),file="main_outcomes.csv",append=TRUE)



In [ ]:
upper = quantile(exp$soda, c(0.025,0.975))[[2]] - change3
lower = change3 - quantile(exp$soda, c(0.025,0.975))[[1]]
p_value = sum(exp$soda >= 0)/1000.0
if (change3 > 0){
    p_value = sum(exp$soda <= 0)/1000.0
}

write(paste(list(race,treatmentVar,'soda',change3,upper,lower,a3,b3,p_value),collapse=', '),file="main_outcomes.csv",append=TRUE)

In [ ]:
upper = quantile(exp$bmi, c(0.025,0.975))[[2]] - change4
lower = change4 - quantile(exp$bmi, c(0.025,0.975))[[1]]
p_value = sum(exp$bmi >= 0)/1000.0
if (change4 > 0){
    p_value = sum(exp$bmi <= 0)/1000.0
}

write(paste(list(race,treatmentVar,'bmi',change4,upper,lower,a4,b4,p_value),collapse=', '),file="main_outcomes.csv",append=TRUE)



In [ ]:
upper = quantile(exp$bmi_abs, c(0.025,0.975))[[2]] - change5
lower = change5 - quantile(exp$bmi_abs, c(0.025,0.975))[[1]]
p_value = sum(exp$bmi_abs >= 0)/1000.0
if (change5 > 0){
    p_value = sum(exp$bmi_abs <= 0)/1000.0
}


write(paste(list(race,treatmentVar,'bmi_abs',change5,upper,lower,a4,b4,p_value,1),collapse=', '),file="main_outcomes.csv",append=TRUE)




In [ ]:
upper = quantile(exp$ow, c(0.025,0.975))[[2]] - change6
lower = change6 - quantile(exp$ow, c(0.025,0.975))[[1]]
paste("('High Income',",change6,",",upper,",",lower,")")
p_value = sum(exp$ow >= 0)/1000.0
if (change6 > 0){
    p_value = sum(exp$ow <= 0)/1000.0
}


write(paste(list(race,treatmentVar,'ow',change6,upper,lower,a6,b6,p_value),collapse=', '),file="main_outcomes.csv",append=TRUE)

